In [ ]:
import getpass
from requests import Session
from pprint import pprint

url = "http://127.0.0.1"
user = input()
password = getpass.getpass()

# LOGIN

In [ ]:
session = Session()
req = session.post(
    f"{url}/auth-token/", 
    data={"username": user, "password": password}
)
if req.status_code != 200:
    print(req.json())
    exit(1)
token = req.json()["token"]
session.headers["Authorization"] = f"Token {token}"

# GET DUMP LIST

In [ ]:
dumps = session.get(f"{url}/api/dumps/").json()
print(f"{len(dumps)} dumps found")
pprint(dumps[0])

# CREATE DUMP

In [ ]:
files = {'upload': open('/home/DATA/AMF_MemorySamples/linux/sorpresa.zip','rb')}
values = {'operating_system': 'Linux', 'name': 'sorpresa'}
res = session.post(f"{url}/api/dumps/", files=files, data=values)
if res.status_code == 200:
    pprint(res.json())
    dump_pk = res.json()["pk"]
else:
    print(res.status_code)

# CREATE DUMP FROM REMOTE DUMP

In [ ]:
# This code requires a file on the server in the folder specified in the LOCAL_UPLOAD_PATH
# settings folder

res = session.post(f"{url}/api/dumps/import_local/", files=(
        ('operating_system', (None, 'Linux')),
        ('name', (None, 'sasf3sfas33')),
        ('filepath', (None, '/uploads/linux/linux-sample-4.bin')),
    ))
if res.status_code == 200:
    pprint(res.json())
else:
    print(res.status_code)

# GET PLUGIN LIST

In [ ]:
res = session.get(f"{url}/api/plugin/")
if res.status_code == 200:
    plugins = res.json()
    print(f"{len(plugins)} plugins found")
    pprint(plugins[0])

# GET PLUGIN STATUS ON SELECTED DUMP

In [ ]:
res = session.get(f"{url}/api/dumps/{dump_pk}/results/")
if res.status_code == 200:
    pprint(res.json())
    result_pk = [x['pk'] for x in res.json() if x['plugin'] == 'linux.pslist.PsList'][0]
    print(res.status_code)

# RESUBMIT PLUGIN

In [ ]:
res = session.post(f"{url}/api/dumps/{dump_pk}/results/{result_pk}/resubmit/", data={'parameter': {'dump': True}})
if res.status_code == 200:
    pprint(res.json())
else:
    print(res.status_code)

# CHECK PLUGIN STATUS

In [ ]:
status = 'Running'
while status != 'Success':
    res = session.get(f"{url}/api/dumps/{dump_pk}/results/{result_pk}/")
    if res.status_code == 200:
        status = res.json()['status']
        pprint(status)
    else:
        print(res.status_code)
        break

# SHOW PLUGIN RESULT

In [ ]:
res = session.get(f"{url}/api/dumps/{dump_pk}/results/{result_pk}/result")
if res.status_code == 200:
    pprint(len(res.json()))
else:
    print(res.status_code)

# WORK WITH RETURNED DATA

In [ ]:
import pandas as pd
import pygwalker as pyg
df = pd.DataFrame.from_records(res.json())
walker = pyg.walk(df)